> for pyspark installed seperately


In [1]:
import findspark
findspark.init()
findspark.find()


'/opt/spark'

In [2]:
# Import PySpark
from pyspark.sql import SparkSession


In [ ]:

# Create SparkSession
spark = SparkSession.builder.appName('Test').getOrCreate()


> read csv


In [ ]:

df = spark.read.option("header", "true").csv("./employees.csv",)
df.printSchema()
# print all rows
df.show(df.count())


In [5]:
# select max salary employee
max_val = df.select(df['Salary'].cast("int")).rdd.max()[0]
df.filter(f"Salary == '{max_val}'").select("First Name", "Last Name").show()


+----------+---------+
|First Name|Last Name|
+----------+---------+
|   Theresa|      Lee|
+----------+---------+



> save by partitioning


In [ ]:

df.select("First Name", "Last Name", "Date of Joining", "Gender").write.option("header", True) \
    .partitionBy("Gender") \
    .mode("overwrite") \
    .csv("partition_gender")


In [6]:
from pyspark.sql.functions import avg, count, min, max, stddev
funcs = {
    "avg": avg,
    "count": count,
    "min": min,
    "max": max,
    "stddev": stddev,
}
for key, val in funcs.items():
    df.select(val(df['Salary'].cast("int")).alias(key)).show()


+---------+
|      avg|
+---------+
|119738.09|
+---------+

+-----+
|count|
+-----+
|  100|
+-----+

+-----+
|  min|
+-----+
|42005|
+-----+

+------+
|   max|
+------+
|197537|
+------+

+-----------------+
|           stddev|
+-----------------+
|46185.27819425479|
+-----------------+



In [7]:

from pyspark.sql.functions import lit, to_date, current_date, col, datediff

# find number of days from joining
df.withColumn('Days from Joining',
              datediff(current_date(), to_date(
                  df['Date of Joining'], 'm/d/yyyy'))
              ).show()


+------+----------+---------+------+--------------------+-------------+---------------+------+-----------+------------+--------------+--------------+-----+-----------------+
|Emp ID|First Name|Last Name|Gender|              E Mail|Date of Birth|Date of Joining|Salary|Last % Hike|  Phone No. |        County|          City|  Zip|Days from Joining|
+------+----------+---------+------+--------------------+-------------+---------------+------+-----------+------------+--------------+--------------+-----+-----------------+
|677509|      Lois|   Walker|     F|lois.walker@hotma...|    3/29/1981|     11/24/2003|168251|        21%|303-572-8492|        Denver|        Denver|80224|             7032|
|940761|    Brenda| Robinson|     F|brenda.robinson@g...|    7/31/1970|      7/27/2008| 51063|        27%|225-945-4954|       De Soto|     Stonewall|71078|             5203|
|428945|       Joe| Robinson|     M|joe.robinson@gmai...|    6/16/1963|       8/3/2016| 50155|        16%|219-904-2161|       Clin